# Process Electricity Consumption

In [1]:
import numpy as np
import pandas as pd

In [4]:
file_path = "../City_Data/GRU_Electric_Data_byJulian/"

premises_df             = pd.read_csv(file_path + "PREMISE.csv")
contracts_df            = pd.read_csv(file_path + "CONTRACT.csv")
installations_df        = pd.read_csv(file_path + "INSTALL.csv")
consumption_df          = pd.read_csv(file_path + "CONSUMPTION.csv")
contract_accounts_df    = pd.read_csv(file_path + "CONTRACT_ACCOUNT.csv")
business_partners_df    = pd.read_csv(file_path + "BUSINESS_PARTNER.csv")
dunning_disconnects_df  = pd.read_csv(file_path + "DUNNING_DISCONNECTS.csv")

# Non-residential contract accounts
non_res_contract_accounts_df = contract_accounts_df[contract_accounts_df['CA_TYPE']=='NR']

# Non-residential contracts
business_contracts_df = contracts_df[contracts_df['CA'].isin(non_res_contract_accounts_df['CA'])]

# Non-residential installs
business_intalls_df = installations_df[installations_df['INSTALL'].isin(business_contracts_df['INSTALL'])]

# Non-residential premises
non_res_premises_df = premises_df[premises_df['PREMISE'].isin(business_intalls_df['PREMISE'])]
non_res_premises_df = non_res_premises_df[(non_res_premises_df['PREMISETYPE'] != 'XTELE') & 
                                          (non_res_premises_df['PREMISETYPE'] != 'RCMUSE')]

# Non-residential consumption
business_consumption_df = consumption_df[consumption_df['CONTRACT'].isin(business_contracts_df['CONTRACT'])]

** Electricity consumption for businesses **

In [5]:
def get_year_and_month(date):
    parts = date.split('/')
    global years, months
    try:
        years.append(int(parts[0]))
        months.append(int(parts[1]))
    except:
        years.append(0)
        months.append(0)

In [6]:
business_contracts_df[:1]

,INSTALL,CONTRACT,CA,STARTDATE,ENDDATE
0,5.000059e+09,4000332116,200023567634,20070425,20070916


In [7]:
contract_accounts_df[:1]

,CA,BP,CA_TYPE,MAILINGADDRESS,MCITY,MREGION,MPOST_CODE,MCOUNTRY
0,200058060632,1000226073,R,12525 SW 11TH AVE,NEWBERRY,FL,32669-3059,US


In [8]:
business_partners_df[:1]

,BP,BP_TYPE,FIRST_NAME,LAST_NAME
0,1000091673,1,GABRIELE,HEIN


In [9]:
business_consumption_df[:1]

,CONTRACT,CONSUMPTION,UNIT,BILLINGPERIOD,BILLSTARTDATE,BILLENDDATE
142674,4001746555,653,KWH,2017/02,20170128,20170224


In [10]:
installations_df[:1]

,INSTALL,PREMISE,INSTALLTYPE,ZZCODE
0,5000001433,7000050491,ELEC,NaN


In [11]:
# Get rid of bad data
bad_values = [
    np.nan,
    '00000-000-000',
    '55555-555-555',
    '66666-666-666',
    '77777-777-777',
    '88888-888-888',
    '99999-999-999',
    'UFCAM-PUS-000',
    'UFCAM-PUS-XXX'
]

premises_df['PARCEL'] = premises_df['PARCEL'] \
    .drop(axis='index', labels=premises_df.index[premises_df['PARCEL'].isin(bad_values)]) \
    .apply(lambda x: x.replace('-', ''))

# Merge datasets around "CONTRACT"
contract_details_df = business_contracts_df \
    .merge(contract_accounts_df[['CA', 'BP']], on='CA', how='inner') \
    .merge(business_partners_df[['BP', 'FIRST_NAME', 'LAST_NAME']], on='BP', how='inner') \
    .merge(business_consumption_df[['CONTRACT', 'CONSUMPTION', 'UNIT', 'BILLINGPERIOD']], on='CONTRACT', how='inner') \
    .merge(installations_df[installations_df['INSTALLTYPE'] == 'ELEC'][['INSTALL', 'PREMISE']], on='INSTALL', how='inner') \
    .merge(premises_df[['PREMISE', 'PARCEL']], on='PREMISE', how='inner')[[
#         'INSTALL',
#         'CONTRACT',
#         'FIRST_NAME',
#         'LAST_NAME',
        'CONSUMPTION',
        'BILLINGPERIOD',
        'PARCEL'
    ]]

contract_details_df

,CONSUMPTION,BILLINGPERIOD,PARCEL
0,11,2007/06,08230004015
1,0,2007/05,08230004015
2,19,2007/07,08230004015
3,399,2007/08,08230004015
4,"1,227",2007/09,08230004015
5,189,2007/09,08230004015
6,16,2013/06,09013005027
7,34,2013/05,09013005027
8,11,2013/04,09013005027
9,57,2013/06,09013005027


In [50]:
years = []
months = []
business_consumption['BILLINGPERIOD'].apply(get_year_and_month)
business_consumption['BILLINGYEAR'] = years
business_consumption['BILLINGMONTH'] = months

/home/asiglani/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/asiglani/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [51]:
tempo = pd.merge(business_consumption, business_contracts, how='inner')
tempo.head()

,CONTRACT,CONSUMPTION,UNIT,BILLINGPERIOD,BILLSTARTDATE,BILLENDDATE,BILLINGYEAR,BILLINGMONTH,INSTALL,CA,STARTDATE,ENDDATE
0,4001746555,653,KWH,2017/02,20170128,20170224,2017,2,5.000366e+09,200039280927,20160830,99991231
1,4001746555,"1,030",KWH,2017/03,20170225,20170329,2017,3,5.000366e+09,200039280927,20160830,99991231
2,4001746555,943,KWH,2017/04,20170330,20170426,2017,4,5.000366e+09,200039280927,20160830,99991231
3,4001746555,917,KWH,2017/05,20170427,20170526,2017,5,5.000366e+09,200039280927,20160830,99991231
4,4001746555,839,KWH,2017/06,20170527,20170628,2017,6,5.000366e+09,200039280927,20160830,99991231


In [52]:
electricity_cons_premisewise = pd.merge(tempo, business_intalls, how='inner')

In [53]:
# ACTIVE BUSINESS DATAFRAME
act_bus = pd.read_csv('../Active_Businesses_with_parcel.csv')

In [173]:
act_bus[act_bus['Parcel'] == '06013004008']

,Unnamed: 0,id,Business_Type,Name,Start_Date,Address,Latitude,Longitude,Parcel
1132,1132,1133,CONTRACTOR-ELECTRICAL,"DELTA TECHNOLOGIES, INC",01/10/2000 12:00:00 AM,"7010 NW 23RD WAY, SUITE B",29.718,-82.356516,06013004008
1816,1816,1817,MANUFACTURING NOT LISTED SEPARATELY,"AFRICAN FORMULA COSMETICS, INC",04/28/2015 12:00:00 AM,"7010 NW 23RD WAY, STE #A",29.718,-82.356516,06013004008


#### GRU Electricity consumption for each active business – Merge Active Businesse

In [67]:
electricity_cons_premisewise.head()

,CONTRACT,CONSUMPTION,UNIT,BILLINGPERIOD,BILLSTARTDATE,BILLENDDATE,BILLINGYEAR,BILLINGMONTH,INSTALL,CA,STARTDATE,ENDDATE,PREMISE,INSTALLTYPE,ZZCODE
0,4001746555,653,KWH,2017/02,20170128,20170224,2017,2,5.00037e+09,200039280927,20160830,99991231,7000095215,FIT,NaN
1,4001746555,"1,030",KWH,2017/03,20170225,20170329,2017,3,5.00037e+09,200039280927,20160830,99991231,7000095215,FIT,NaN
2,4001746555,943,KWH,2017/04,20170330,20170426,2017,4,5.00037e+09,200039280927,20160830,99991231,7000095215,FIT,NaN
3,4001746555,917,KWH,2017/05,20170427,20170526,2017,5,5.00037e+09,200039280927,20160830,99991231,7000095215,FIT,NaN
4,4001746555,839,KWH,2017/06,20170527,20170628,2017,6,5.00037e+09,200039280927,20160830,99991231,7000095215,FIT,NaN


In [56]:
elec_demo = electricity_cons_premisewise.merge(non_res_premises,how='inner',on='PREMISE')

In [68]:
elec_demo.head()["PARCEL"]

0    16241-003-000
1    16241-003-000
2    16241-003-000
3    16241-003-000
4    16241-003-000
Name: PARCEL, dtype: object

In [74]:
elec_demo['PARCEL'] = elec_demo['PARCEL'].apply(lambda x: x.replace('-','') if type(x) == str else x)

In [58]:
#elec_demo.set_index(['PARCEL','CONTRACT','INSTALL','BILLINGYEAR','BILLINGMONTH'])

In [87]:
elec_consuption_gnv = act_bus[['Name','Parcel']].dropna(subset=['Parcel']).merge(elec_demo[['PARCEL','CONSUMPTION','BILLINGYEAR']].dropna(subset=['PARCEL']), how='left', left_on='Parcel', right_on='PARCEL')#.set_index(['Parcel','BILLINGYEAR', 'BILLINGMONTH'])

In [155]:
exp = elec_consuption_gnv.dropna(subset=['Parcel','PARCEL']).sort_values(by=['Parcel','BILLINGYEAR'])

In [133]:
import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')

'en_US.UTF-8'

In [157]:
def stringToFloat(x):
    if(type(x)==str):
        return atof(x)
    return x

In [158]:
exp['CONSUMPTION'] = exp['CONSUMPTION'].apply(stringToFloat)

In [162]:
elec_cons_parcel = exp[['Name','Parcel','BILLINGYEAR','CONSUMPTION']].groupby(['Name','Parcel','BILLINGYEAR'],as_index=False).mean()

In [164]:
writer = pd.ExcelWriter('ELectricity_consumption_with_parcelinfo.xlsx')
elec_cons_parcel.to_excel(writer, 'Sheet1')
writer.save()

** Disconnects at a given premise **

In [46]:
dunning_demo = dunning_disconnects.merge(business_partners, how='inner').merge(business_contracts).merge(business_intalls).merge(non_res_premises)

In [47]:
dunning_demo.set_index(['PARCEL','PREMISE','CA'])

BP        DISCNO  DISC_DATE  \
PARCEL      PREMISE    CA                                                  
09925001000 7000017189 200036591094  1000123638  810000265198   20120201   
                       200043093734  1000168199  810000440944   20150804   
08796000000 7000064629 200032665830  1000041016  810000130230   20090609   
                       200032665830  1000041016  810000130230   20090609   
                       200032665830  1000041016  810000130230   20090609   
                       200032665830  1000041016  810000130230   20090609   
                       200032665830  1000041016  810000130230   20090609   
08199046000 7000057079 200035281901  1000063661  810000258335   20111205   
                       200035281901  1000063661  810000276708   20120517   
                       200035281901  1000063661  810000268736   20120306   
                       200035281901  1000063661  810000454681   20151120   
                       200035281901  1000063661  810000161664   20100107   
                       200012804775  1000063661  810000149077   20091006   
                       200012804775  1000063661  810000073590   20080604   
nan         7000059672 200015777423  1000013093  810000029223   20070820   
                       200027279906  1000105384  810000151014   20091021   
06860366101 7000100094 200024209450  1000013093  810000055333   20080122   
11879000000 7000087865 200020585993  1000040046  810000140294   20090803   
                       200020585993  1000040046  810000140294   20090803   
                       200020585993  1000040046  810000102534   20081201   
                       200020585993  1000040046  810000102534   20081201   
                       200020585993  1000040046  810000165866   20100208   
                       200020585993  1000040046  810000165866   20100208   
                       200020585993  1000040046  810000152373   20091029   
                       200020585993  1000040046  810000152373   20091029   
                       200020585993  1000040046  810000140294   20090803   
                       200020585993  1000040046  810000140294   20090803   
                       200020585993  1000040046  810000102534   20081201   
                       200020585993  1000040046  810000102534   20081201   
                       200020585993  1000040046  810000165866   20100208   
...                                         ...           ...        ...   
06394042000 7000014501 200051236175  1000055443  810000472894   20160418   
                       200051236175  1000055443  810000472894   20160418   
                       200051236175  1000055443  810000472894   20160418   
                       200051236175  1000055443  810000472894   20160418   
                       200051236175  1000055443  810000472894   20160418   
            7000126712 200051236175  1000055443  810000472894   20160418   
nan         7000053684 200035660706  1000049996  810000540665   20170629   
                       200035660706  1000049996  810000540665   20170629   
                       200035660706  1000049996  810000540665   20170629   
07240024000 7000028604 200045312004  1000178178  810000542532   20170710   
                       200045312004  1000178178  810000542532   20170710   
                       200045312004  1000178178  810000542532   20170710   
07966001012 7000048746 200053000060  1000134556  810000544983   20170725   
06233000000 7000002072 200055919760  1000219278  810000545827   20170731   
14649000000 7000049775 200044692618  1000176140  810000548951   20170815   
07882003002 7000002774 200061198984  1000050426  810000550882   20170830   
                       200061198984  1000050426  810000550882   20170830   
                       200061198984  1000050426  810000550882   20170830   
07967010001 7000107306 200052378553  1000205271  810000544976   20170725   
14518002000 7000089808 200054049579  1000211282  810000542190   20170710   
                 

In [50]:
act_bus.merge(dunning_demo, how='left', left_on='Parcel', right_on='PARCEL').set_index(['PREMISE','PARCEL'])

Unnamed: 0    id  \
PREMISE      PARCEL                          
NaN          NaN                   0     1   
             NaN                   1     2   
             NaN                   2     3   
7.000072e+09 08199047000           3     4   
NaN          NaN                   4     5   
             NaN                   5     6   
             NaN                   6     7   
             NaN                   7     8   
             NaN                   8     9   
             NaN                   9    10   
             NaN                  10    11   
             NaN                  11    12   
             NaN                  12    13   
             NaN                  13    14   
             NaN                  14    15   
             NaN                  15    16   
             NaN                  16    17   
             NaN                  17    18   
             NaN                  18    19   
             NaN                  19    20   
             NaN                  20    21   
7.000057e+09 08199092000          21    22   
             08199092000          21    22   
             08199092000          21    22   
             08199092000          21    22   
NaN          NaN                  22    23   
             NaN                  23    24   
             NaN                  24    25   
             NaN                  25    26   
7.000006e+09 08681000000          26    27   
...                              ...   ...   
NaN          NaN                5007  5008   
             NaN                5008  5009   
             NaN                5009  5010   
             NaN                5010  5011   
             NaN                5011  5012   
             NaN                5012  5013   
7.000047e+09 08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
7.000098e+09 08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
             08035005000        5013  5014   
7.000015e+09 07878020016        5014  5015   
             07878020016        5014  5015   
             07878020016        5014  5015   
             07878020016        5014  5015   
NaN          NaN                5015  5016   
             NaN                5016  5017   
             NaN                5017  5018   
             NaN                5018  5019   

                                                    Business_Type  \
PREMISE      PARCEL                                                 
NaN          NaN                               AUTO REPAIR GARAGE   
             NaN                                  RETAIL MERCHANT   
             NaN                         STORAGE WAREHOUSE OR LOT   
7.000072e+09 08199047000      REPAIRS NOT REQUIRING STATE LICENSE   
NaN          NaN                                    TRAVEL AGENCY   
             NaN                    CONTRACTOR-HARV CLASS A UNLTD   
             NaN                                  RETAIL MERCHANT   
             NaN                                  RETAIL MERCHANT   
             NaN                                    BEAUTY PARLOR   
             NaN                                  RETAIL MERCHANT   
             NaN                               WHOLESALE MERCHANT   
             NaN                EMPLOYMENT & TEMP SERVICES AGENCY   
             NaN                                           TAILOR   
             NaN                                  RETAIL MERCHANT   
             NaN       

### Change of contracts at given location – Change of businesses 

In [23]:
tempo1 = business_contracts.merge(business_intalls).merge(non_res_premises)#.set_index(['PREMISE','CA','CONTRACT'])

In [24]:
tempo1

,INSTALL,CONTRACT,CA,STARTDATE,ENDDATE,PREMISE,INSTALLTYPE,ZZCODE,SERVICEADDRESS,SCITY,SREGION,SPOST_CODE,SCOUNTRY,TAXJURCODE,PARCEL,PREMISETYPE
0,5.00006e+09,4000332116,200023567634,20070425,20070916,7000014285,ELEC,NaN,1221 NE 28TH AVE,GAINESVILLE,FL,32609,US,FLALA0004,08230-004-015,RSFD
1,5.00025e+09,4000335421,200023695350,20070508,20090629,7000071867,ELEC,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
2,5.00025e+09,4000335424,200023695350,20070508,20090629,7000071867,WATR,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
3,5.00025e+09,4000335425,200023695350,20070508,20090629,7000071867,WSTU,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
4,5.00025e+09,4000335422,200023695350,20070508,20090629,7000071867,NTGS,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
5,5.00025e+09,4000335423,200023695350,20070508,20090629,7000071867,STRM,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
6,5.00025e+09,4000335426,200023695350,20070508,20090629,7000071867,REFS,NaN,3835 NW 8TH AVE,GAINESVILLE,FL,32605,US,FLALA0004,06487-014-006,RSFD
7,5.00034e+09,4000345519,200023673728,20070529,20071211,7000101276,ELEC,NaN,4444 NW 82ND RD,GAINESVILLE,FL,32653,US,FLALA0004,NaN,XTEMP
8,5.00034e+09,4000448934,200023673728,20071212,20080626,7000101276,ELEC,NaN,4444 NW 82ND RD,GAINESVILLE,FL,32653,US,FLALA0004,NaN,XTEMP
9,5.00034e+09,4000348093,200023921985,20070601,20070827,7000097267,ELEC,NaN,4537 NE 16TH TER,GAINESVILLE,FL,32609,US,FLALA0004,08160-010-023,XTEMP


In [25]:
tempo1[tempo1['INSTALLTYPE']=='ELEC'].merge(non_res_contract_acnts).set_index(['PREMISE','CA','CONTRACT'])

INSTALL  STARTDATE   ENDDATE  \
PREMISE    CA           CONTRACT                                       
7000014285 200023567634 4000332116  5.00006e+09   20070425  20070916   
7000071867 200023695350 4000335421  5.00025e+09   20070508  20090629   
7000101276 200023673728 4000345519  5.00034e+09   20070529  20071211   
                        4000448934  5.00034e+09   20071212  20080626   
7000097162 200023673728 4000479502  5.00034e+09   20080324  20080428   
7000101317 200023673728 4000448935  5.00035e+09   20071212  20080321   
7000097267 200023921985 4000348093  5.00034e+09   20070601  20070827   
7000093191 200024148220 4000350533  5.00009e+09   20070608  20070712   
7000042306 200014123167 4000352707  5.00034e+09   20070609  20080514   
7000097257 200014123167 4000139228  5.00014e+09   20070407  20080514   
7000019862 200024426789 4000360312  5.00017e+09   20070702  20080916   
7000075667 200024441947 4000360541   5.0002e+09   20070629  20071001   
7000098942 200024811961 4000381598  5.00034e+09   20070725  20081001   
7000100293 200025137519 4000389772  5.00034e+09   20070730  20071129   
7000099409 200025137519 4000444372  5.00035e+09   20071130  20080302   
7000099497 200025297769 4000403894  5.00034e+09   20070808  20080629   
7000099498 200025297870 4000403897  5.00034e+09   20070808  20080629   
7000099494 200025296254 4000403900  5.00034e+09   20070808  20070909   
7000099493 200025296153 4000403939  5.00034e+09   20070808  20070826   
7000099418 200025200971 4000403942  5.00034e+09   20070808  20071108   
7000099417 200025200971 4000438573  5.00034e+09   20071109  20080211   
7000099416 200025200466 4000403943  5.00034e+09   20070808  20071108   
7000099415 200025200466 4000438569  5.00034e+09   20071109  20080214   
7000099491 200025295951 4000403944  5.00034e+09   20070808  20070904   
7000099499 200025297971 4000403945  5.00034e+09   20070808  20080826   
7000099602 200025300395 4000403948  5.00034e+09   20070808  20080505   
7000099496 200025297668 4000403957  5.00034e+09   20070808  20070911   
7000099604 200025300597 4000403963  5.00034e+09   20070808  20070823   
7000099603 200025300496 4000403966  5.00034e+09   20070808  20080707   
7000099606 200025300803 4000403969  5.00034e+09   20070808  20071206   
...                                         ...        ...       ...   
7000100861 200027037204 4000444914  5.00035e+09   20071203  99991231   
7000052296 200062843944 4001880096  5.00014e+09   20170826  99991231   
           200016015677 4000133196  5.00014e+09   20070328  20170825   
7000137341 200062384812 4001863335  5.00039e+09   20170801  99991231   
7000060378 200062616194 4001871370  5.00023e+09   20170811  99991231   
7000137374 200062735325 4001877614  5.00039e+09   20170818  99991231   
7000136152 200061391570 4001822968  5.00039e+09   20170607  99991231   
7000136154 200061391671 4001823025  5.00039e+09   20170607  99991231   
7000136155 200061391772 4001823026  5.00039e+09   20170607  99991231   
7000133413 200062579721 4001871413  5.00039e+09   20170810  99991231   
           200057660912 4001826224  5.00039e+09   20170614  20170809   
7000137362 200062578812 4001872691  5.00039e+09   20170810  99991231   
7000131728 200011311177 4001627063  5.00031e+09   20160115  99991231   
7000137453 200062820302 4001880434  5.00039e+09   20170825  99991231   
7000035086 200012062020 4000191105   5.0002e+09   20070329  99991231   
7000137367 200062684195 4001875498  5.00039e+09   20170815  99991231   
7000136011 200060885150 4001794920  5.00039e+09   20170324  20170815   
7000137372 200062581135 4001878808  5.00039e+09   20170822  99991231   
7000137381 200062821918 4001880376  5.00039e+09   20170825  99991231   
7000137557 200062820504 4001880823  5.00039e+09   20170828  99991231   
7000137560 200062820706 4001880824  5.00039e+09   20170828  99991231   
7000137556 200062820403 4001880717  5.00039e+09   20170828  99991231   
7000137565 200062831820 4001872516  5.00039e+09   20170828  99991231   
70001375

** Business Premises not in active businesses **

In [26]:
act_bus = pd.read_csv('../Active_Businesses_with_parcel.csv')

In [27]:
non_res_premises['PARCEL'] = non_res_premises['PARCEL'].apply(lambda x: str(x).replace('-',''))

In [28]:
non_res_premises[non_res_premises['PARCEL'].isin(act_bus['Parcel'])].count()

PREMISE           3367
SERVICEADDRESS    3367
SCITY             3367
SREGION           3367
SPOST_CODE        3367
SCOUNTRY          3367
TAXJURCODE        3367
PARCEL            3367
PREMISETYPE       3338
dtype: int64

In [29]:
# Trying to find installs for a particular premise with address '2930 SW 23RD TER'

business_intalls[business_intalls['PREMISE']==7000124123]    # -- Here Premise number is 7000124123
business_contracts[business_contracts['INSTALL']==5000376194] # For above premise here install no is 5000376194

,INSTALL,CONTRACT,CA,STARTDATE,ENDDATE
1210079,5.000376e+09,4001305074,200048721249,20131010,99991231


In [30]:
# Trying to find installs for a particular premise with address '2930 SW 23RD TER'

business_intalls[business_intalls['PREMISE']==7000124124] # -- Here Premise number is 7000124124
business_contracts[business_contracts['INSTALL']==5000376195] # For above premise here install no is 5000376195

,INSTALL,CONTRACT,CA,STARTDATE,ENDDATE
1210083,5.000376e+09,4001305263,200048721350,20131010,99991231


In [ ]:
# non_res_contract_acnts[non_res_contract_acnts['CA']==200048721249]

# non_res_contract_acnts[non_res_contract_acnts['CA']==200048904438]

In [101]:
non_res_premises[non_res_premises['SERVICEADDRESS'] == '2930 SW 23RD TER ']

,PREMISE,SERVICEADDRESS,SCITY,SREGION,SPOST_CODE,SCOUNTRY,TAXJURCODE,PARCEL,PREMISETYPE
94845,7000124123,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
94846,7000124124,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96495,7000123951,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96547,7000124221,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96548,7000124222,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96549,7000124223,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96553,7000124229,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96555,7000124300,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD
96556,7000124301,2930 SW 23RD TER,GAINESVILLE,FL,32608,US,FLALA0004,07314-000-000,CNOBLD


In [179]:
business_dunning_disconnects = dunning_disconnects[dunning_disconnects['CA'].isin(business_contracts['CA'])]

In [182]:
business_dunning_disconnects

,BP,CA,DISCNO,DISC_DATE
0,1000123638,200036591094,810000265198,20120201
433,1000051933,200012249956,810000068502,20080430
437,1000003144,200012433650,810000072872,20080530
810,1000012073,200044023520,810000300765,20121101
1426,1000037374,200019577702,810000044622,20071108
1514,1000088290,200011903786,810000066818,20080421
1672,1000009822,200031182942,810000122057,20090409
2086,1000124006,200035463066,810000215458,20110111
2415,1000104773,200027089643,810000175895,20100419
2454,1000009612,200014616655,810000224736,20110330


In [186]:
business_contracts[business_contracts['CA'] == 200044023520]

,INSTALL,CONTRACT,CA,STARTDATE,ENDDATE
1284272,5.000055e+09,4001105785,200044023520,20120711,20130807
1290450,5.000055e+09,4001105786,200044023520,20120711,20130807
1295576,5.000055e+09,4001105787,200044023520,20120711,20130807


In [188]:
business_contracts[business_contracts['CA'] == 200044023520]['INSTALL'][1284272]

5000055299.0

In [189]:
business_intalls[business_intalls['INSTALL'] == 5000055299]

,INSTALL,PREMISE,INSTALLTYPE,ZZCODE
332939,5000055299,7000057245,ELEC,NaN


In [190]:
business_intalls[business_intalls['PREMISE']==7000057245]

,INSTALL,PREMISE,INSTALLTYPE,ZZCODE
313438,5000055320,7000057245,WSTU,NaN
318542,5000055313,7000057245,WATR,NaN
319651,5000060108,7000057245,IRRW,NaN
332939,5000055299,7000057245,ELEC,NaN


In [185]:
non_res_contract_acnts[non_res_contract_acnts['CA'] == 200012433650]

,CA,BP,CA_TYPE,MAILINGADDRESS,MCITY,MREGION,MPOST_CODE,MCOUNTRY
204398,200012433650,1000003144,NR,501 SE 18TH ST OFC,GAINESVILLE,FL,32641-7455,US


In [159]:
ins[464490]
ab = int(ins[464490])
ab

5000363121

In [162]:
business_intalls[business_intalls['INSTALL'] == ab]

,INSTALL,PREMISE,INSTALLTYPE,ZZCODE
372246,5000363121,7000111811,GRUC,NaN


In [128]:
unique_premises = business_intalls['PREMISE'].unique()